# QUẢN LÝ DỮ LIỆU CỔNG (Thay thế PgAdmin4)

Đây là file Notebook (.ipynb) giúp bạn quản lý database **SQLite** một cách nhanh chóng, trực quan ngay trong VS Code

### Hướng dẫn:
1. Đảm bảo đã chạy `npm install sqlite3` (cần cho server, không cần cho notebook này).
2. Nhấn nút **Run All**.
3. **Lưu ý:** Nếu chưa chạy `run_app.bat`, database sẽ chưa có dữ liệu.

In [3]:
import sqlite3
import pandas as pd
from IPython.display import display, HTML
import os

# 1. KẾT NỐI DATABASE (Tự động tạo nếu chưa có)
db_path = 'gate_system.db'
conn = sqlite3.connect(db_path)

# --- FIX LỖI: Tự tạo bảng 'logs' nếu server chưa chạy ---
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
        event_type TEXT,
        description TEXT
    )
''')
conn.commit()
# --------------------------------------------------------

print(f" Đã kết nối tới: {db_path} (và đảm bảo bảng 'logs' tồn tại)")

 Đã kết nối tới: gate_system.db (và đảm bảo bảng 'logs' tồn tại)


### 1. Xem dữ liệu mới nhất (Live Data View)

In [4]:
# Đọc 10 dòng mới nhất
try:
    query = "SELECT * FROM logs ORDER BY id DESC LIMIT 10"
    df = pd.read_sql_query(query, conn)
    
    if df.empty:
        print("Chưa có dữ liệu!")
    else:
        # Nếu cột timestamp là dạng text chuẩn, pandas sẽ hiển thị đúng
        display(df)
except Exception as e:
    print(f"Lỗi: {e}")

,id,timestamp,event_type,description
0,316,2025-12-18 10:53:36,CAR_IN,50T75814 - IN (MQTT)
1,315,2025-12-18 10:53:32,CAR_IN,50S874 - IN (MQTT)
2,314,2025-12-18 10:53:17,CAR_IN,57I5874 - IN (MQTT)
3,313,2025-12-18 10:53:07,CAR_IN,3UA5814 - IN (MQTT)
4,312,2025-12-18 10:50:57,CAR_IN,30A237 - IN (MQTT)
5,311,2025-12-18 10:50:55,CAR_IN,30A777 - IN (MQTT)
6,310,2025-12-18 10:50:49,CAR_IN,80A77777 - IN (MQTT)
7,309,2025-12-18 10:50:46,CAR_IN,35Z77777 - IN (MQTT)
8,308,2025-12-18 10:50:00,CAR_IN,30A27777 - IN (MQTT)
9,307,2025-12-18 10:49:33,CAR_IN,22A77772 - IN (MQTT)


### 2. Thống kê nhanh
Đếm số lần mở cổng và nhận diện biển số.

In [5]:
try:
    stats_query = """
    SELECT event_type, COUNT(*) as count 
    FROM logs 
    GROUP BY event_type
    """
    df_stats = pd.read_sql_query(stats_query, conn)
    if not df_stats.empty:
        display(df_stats)
    else:
        print(" Chưa có dữ liệu để thống kê.")
except Exception as e:
    print(f" Lỗi: {e}")

,event_type,count
0,CAR_IN,68
1,CAR_OUT,39
2,GATE_IN_OFF,35
3,GATE_IN_ON,44
4,GATE_OUT_OFF,31
5,GATE_OUT_ON,50
6,GATE_STATUS,31
7,LICENSE_PLATE_ENTRY,6
8,LICENSE_PLATE_EXIT,12


### 3. Xóa dữ liệu cũ (Cleanup)
Chạy ô này nếu muốn xóa sạch database để test lại.

In [8]:
# Bỏ comment dòng dưới để chạy xóa
# cursor = conn.cursor()
# cursor.execute("DELETE FROM logs")
# conn.commit()
# print(" Đã xóa toàn bộ dữ liệu log!")